In [ ]:
from pathlib import Path
from dummy_problems.dataloaders import LettersDataModule
from dummy_problems.models import MODEL_TYPES, SETTINGS_DL
import lightning as L

In [ ]:
# Train ConvNet
def train_convnet():
    settings =  {
        "num_classes": 26,
        "dataset_dir": Path("/home/ubuntu/data/letters_dataset"),

        "model_type": "DL",
        "model_name": "ConvNet",
        "stage": "fit",
    }
    settings.update(SETTINGS_DL[settings["model_name"]])
    callbacks=[L.pytorch.callbacks.EarlyStopping(monitor="val_loss", mode="min")]

    data = LettersDataModule(settings)
    model = MODEL_TYPES[settings['model_type']](settings)
    trainer = L.Trainer(max_epochs=10, callbacks=callbacks, log_every_n_steps=5)
    trainer.fit(model, data)

train_convnet()

In [2]:
# Test ConvNet
def test_convnet(dataset_size=10):
    settings =  {
        "num_classes": 26,
        "dataset_dir": Path("/home/ubuntu/data/letters_dataset"),

        "model_type": "DL",
        "model_name": "ConvNet",
        "stage": "fit",
        "checkpoint": f"weights/convnet_{dataset_size}.ckpt",
    }

    data = LettersDataModule(settings)
    model = MODEL_TYPES[settings['model_type']].load_from_checkpoint(settings['checkpoint'])
    trainer = L.Trainer()
    trainer.test(model, data)

In [ ]:
# samples per letter = 10 (train + validate)
test_convnet(dataset_size=10)

# samples per letter = 100 (train + validate)
test_convnet(dataset_size=100)

# samples per letter = 1000 (train + validate)
test_convnet(dataset_size=1000)